# Max 2-Sat

In [14]:
#!pip install dwave-system
#!pip install dwave-neal

In [30]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from neal import SimulatedAnnealingSampler
import networkx as nx
import dimod

import dwavebinarycsp


import matplotlib
%matplotlib inline

from matplotlib import pyplot as plt
import numpy as np

In [16]:
# parameters

TOKEN = 'DEV-dadab771e1ad528a25146defb4c58b1e0fe50b33'
PATH = 'testing_sat/02-sat.txt'
NUM_READS_SA = 20
NUM_READS_QPU = 20

# Create QUBO

In [17]:
with open(PATH, "r") as f:
    sat = f.readlines()


data = sat[0].split(" ")

n_variables, n_clauses = int(data[2]), int(data[3])

#print(n_variables)
#print(n_clauses) 

Q = []
c = 0

sat = sat[1:]
clauses = []
for x in sat:
    clauses.append(x.replace(' 0\n', ''))
clauses


    

['1 2',
 '1 -2',
 '-1 2',
 '-1 -2',
 '-1 3',
 '-1 -3',
 '2 -3',
 '2 4',
 '-2 3',
 '-2 -3',
 '3 4',
 '-3 -4']

In [18]:
import numpy as np

def gen_q(clauses):
    Q = np.zeros(shape=[n_variables, n_variables])
    c = 0
    for clause in clauses:
        clause = clause.split(' ')
        int_clause = [int(c) for c in clause]

        s1, s2 = int_clause[0], int_clause[1]
        v1, v2 = abs(s1)-1, abs(s2)-1
        

        if s1 > 0 and s2 > 0: # True True
            # 1 - x1 - x2 + x1x2
            c += 1
            Q[v1][v1] += -1 
            Q[v2][v2] += -1
            Q[v1][v2] += 1/2 
            Q[v2][v1] += 1/2
        elif s1 > 0 and s2 < 0: # True False
            # x2 - x1x2
            Q[v2][v2] += 1
            Q[v1][v2] += -1/2 
            Q[v2][v1] += -1/2

        elif s1 < 0 and s2 > 0: # False True
            # x1 - x1x2
            Q[v1][v1] += 1
            Q[v1][v2] += -1/2 
            Q[v2][v1] += -1/2

        elif s1 < 0 and s2 < 0: # False False
            # x1x2
            Q[v1][v2] += 1/2 
            Q[v2][v1] += 1/2

        else:
            pass # throw error
            
    return Q, c

## Create BQM

In [19]:
Q, c = gen_q(clauses)

variable_order = ["x_{}".format(n) for n in range(1, n_variables+1)]
BQM = dimod.BinaryQuadraticModel.from_numpy_matrix(Q, variable_order = variable_order)
BQM

BinaryQuadraticModel({x_1: 1.0, x_2: 0.0, x_3: 0.0, x_4: -2.0}, {('x_2', 'x_3'): -1.0, ('x_2', 'x_4'): 1.0, ('x_3', 'x_4'): 2.0}, 0.0, 'BINARY')

# Solve

## Simulated Annealing

In [35]:
def simulated_annealing(bqm, num_reads_sa=NUM_READS_SA):
    # Run the QUBO on the solver from your config file
    sampler = SimulatedAnnealingSampler()
    response_SA = sampler.sample(bqm, num_reads=num_reads_sa)
    return response_SA

In [36]:
def print_response_data(response):
    # ------- Print results to user -------
    print('-' * 160)
    print('{:>40s}{:>40s}{:^40s}{:^40s}'.format('Set 0','Set 1','Energy',"Count"))
    print('-' * 160)
    for sample, E, occ in response.data(fields=['sample','energy',"num_occurrences"]):
        S0 = [k for k,v in sample.items() if v == 0]
        S1 = [k for k,v in sample.items() if v == 1]
        print('{:>40s}{:>40s}{:^40s}{:^40s}'.format(str(S0),str(S1),str(E),str(occ)))

In [37]:
print_response_data(response_SA)

----------------------------------------------------------------------------------------------------------------------------------------------------------------
                                   Set 0                                   Set 1                 Energy                                  Count                  
----------------------------------------------------------------------------------------------------------------------------------------------------------------
                   ['x_1', 'x_2', 'x_3']                                 ['x_4']                  -2.0                                     1                    
                   ['x_1', 'x_2', 'x_3']                                 ['x_4']                  -2.0                                     1                    
                   ['x_1', 'x_2', 'x_3']                                 ['x_4']                  -2.0                                     1                    
                   ['x_1', 'x_2', 

In [38]:
print_response_data(response_SA.aggregate())

----------------------------------------------------------------------------------------------------------------------------------------------------------------
                                   Set 0                                   Set 1                 Energy                                  Count                  
----------------------------------------------------------------------------------------------------------------------------------------------------------------
                   ['x_1', 'x_2', 'x_3']                                 ['x_4']                  -2.0                                     20                   


## Real Annealing

In [39]:
# config API

from dwave.cloud import Client


client = Client.from_config(token=TOKEN)     
solver = client.get_solvers()[2]
solver   

StructuredSolver(id='Advantage_system1.1')

In [40]:
import random
from dwave.cloud import Client

def real_annealing(bqm, num_reads_qpu=NUM_READS_QPU, token=TOKEN):
    # Run the QUBO on the solver from your config file
    sampler = SimulatedAnnealingSampler()
    sampler = EmbeddingComposite(DWaveSampler(token=TOKEN))

    response_QPU = sampler.sample(BQM, num_reads=num_reads_qpu)
    return response_QPU

In [26]:
print_response_data(response_QPU)

----------------------------------------------------------------------------------------------------------------------------------------------------------------
                                   Set 0                                   Set 1                 Energy                                  Count                  
----------------------------------------------------------------------------------------------------------------------------------------------------------------
                   ['x_1', 'x_2', 'x_3']                                 ['x_4']                  -2.0                                     20                   


# Results

In [27]:
# retrieve result with the best counts

def return_solution(response, Q, c):
    count = 0
    for i in range(len(response)):
        new_count = response[i][2]
        if  new_count > count:
            best = response[i][0]
            count = new_count
    
    y = c + np.matmul(np.matmul(best.T, Q), best) 

    return y, best

y, best = return_solution(response_SA.aggregate().record, Q, c)
print(f'y is {y}')

for v in range(n_variables):
    print(f'x_{v+1} = {best[v]}')



y is 1.0
x_1 = 0
x_2 = 0
x_3 = 0
x_4 = 1


## Simulated Annealing

# Testing

In [31]:


# define the csp
csp = dwavebinarycsp.factories.random_2in4sat(4, 2) # 8 variables, 4 clauses
csp

# generate the bqm from the csp
import warnings
np.warnings.filterwarnings("ignore", category=DeprecationWarning)

bqm = dwavebinarycsp.stitch(csp)
bqm

BinaryQuadraticModel({2: 4.0, 0: 4.0, 1: -4.0, 3: -4.0}, {(2, 0): -8.0, (2, 1): 0.0, (2, 3): 0.0, (0, 1): 0.0, (0, 3): 0.0, (1, 3): 8.0}, 4.0, 'BINARY')

BinaryQuadraticModel({0: -4.0, 1: 4.0, 2: 4.0, 3: -4.0}, {(0, 1): 0.0, (0, 2): 0.0, (0, 3): 8.0, (1, 2): -8.0, (1, 3): 0.0, (2, 3): 0.0}, 4.0, 'BINARY')